In [140]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import dash

In [141]:
#loading data
crs_data = pd.read_csv("crs_data.csv")

In [142]:
#understanding variables
#crs_data.head()
#crs_data.info()

In [143]:
#figuring out what the unique donor names are, to categorise
#unique_donor_name = crs_data["donor_name"].unique()
#print(unique_donor_name)
#unique_donors.to_csv('unique_donors_250517.csv', index=False)

In [144]:
#Grouping donors into mutually exclusive categories - G7, BRICS, UN, World Bank, Other - etc

g7 = ["United Kingdom", 
      "United States", 
      "France", 
      "Germany", 
      "Italy", 
      "Canada", 
      "Japan"]

brics = ["Brazil", 
         "Russia", 
         "India",
         "China", 
         "South Africa"] # no BRIC countries currently in dataset

eu_institutions = ["EU Institutions"]

foundation = ["Wellcome Trust",
            "IKEA Foundation",
            "William and Flora Hewlett Foundation",
            "Jacobs Foundation",
            "German Postcode Lottery",
            "Open Society Foundations",
            "UBS Optimus Foundation",
            "Susan T. Buffett Foundation",
            "World Diabetes Foundation",
            "Gates Foundation",
            "Rockefeller Foundation",
            "Laudes Foundation",
            "H&M Foundation",
            "David and Lucile Packard Foundation",
            "LEGO Foundation",
            "Children's Investment Fund Foundation",
            "Conrad N. Hilton Foundation",
            "Ford Foundation",
            "Good Ventures Foundation",
            "Citi Foundation",
            "Bloomberg Family Foundation",
            "Arcadia Fund"
            ]

un_agency = ["International Centre for Genetic Engineering and Biotechnology [ICGEB]",
            "WFP",
            "United Nations Industrial Development Organization [UNIDO]",
            "UNICEF",
            "UNFPA",
            "UNHCR",
            "UNDP",
            "UNAIDS",
            "UN Women",
            "International Labour Organisation [ILO]",
            "Joint Sustainable Development Goals Fund [Joint SDG Fund]",
            "UN Development Coordination Office",
            "Food and Agriculture Organisation [FAO]",
            "International Atomic Energy Agency [IAEA]",
            "Central Emergency Response Fund [CERF]",
            "IFAD",
            "COVID-19 Response and Recovery Multi-Partner Trust Fund [UN COVID-19 MPTF]"
            ]

world_bank = ["International Development Association [IDA]",
              "International Bank for Reconstruction and Development [IBRD]"]

other_dfi_ifi_bank = ["OPEC Fund for International Development [OPEC Fund]",
                "Global Fund",
                "Green Climate Fund [GCF]"
                "Private Infrastructure Development Group",
                "Global Environment Facility [GEF]",
                "Adaptation Fund",
                "Nordic Development Fund [NDF]",
                "Islamic Development Bank [IsDB]",
                "Asian Infrastructure Investment Bank [AIIB]",
                "Asian Development Bank [AsDB]",
                "IMF Concessional Trust Funds"
                ]

other_igo_ngo_ppp = ["World Health Organisation [WHO]",
                "World Trade Organisation",
                "WTO - International Trade Centre [ITC]",
                "WHO-Strategic Preparedness and Response Plan [SPRP]",
                "Global Alliance for Vaccines and Immunization [GAVI]"
                ]

other_country = ["Australia",
            "Austria",
            "Azerbaijan",
            "Belgium",
            "Bulgaria",
            "Croatia",
            "Czechia",
            "Denmark",
            "Estonia",
            "Finland",
            "Greece",
            "Hungary",
            "Iceland",
            "Ireland",
            "Israel",
            "Kazakhstan",
            "Korea",
            "Kuwait",
            "Latvia",
            "Liechtenstein",
            "Lithuania",
            "Luxembourg",
            "Malta",
            "Netherlands",
            "New Zealand",
            "Norway",
            "Poland",
            "Portugal",
            "Qatar",
            "Romania",
            "Saudi Arabia",
            "Slovak Republic",
            "Slovenia",
            "Spain",
            "Sweden",
            "Switzerland",
            "Türkiye",
            "Thailand",
            "United Arab Emirates"
]



In [145]:
# define function to apply donor grouping
def donor_group_function (donor):
    if donor in g7:
        return "G7"
    elif donor in brics:
        return "BRICS"
    elif donor in eu_institutions:
        return "EU Institutions"
    elif donor in foundation:
        return "Foundation"
    elif donor in un_agency:
        return "UN Agency"
    elif donor in world_bank:
        return "World Bank"
    elif donor in other_country:
        return "Other - Country"
    elif donor in other_dfi_ifi_bank:
        return "Other - DFI, IFI, Bank"
    else:
        return "Other - IGO, NGO, PPP"
    
#apply function crs_data to a new variable called 'donor group'
crs_data["donor_group"] = crs_data["donor_name"].apply(donor_group_function)

In [146]:
#grouping dataset, calculating sum and rounding to billion - for charts
crs_grouped = crs_data.groupby(['year','donor_group'], as_index=False)['value'].sum()
crs_grouped["value_bn"] = crs_grouped["value"] / 1000
crs_grouped["value_bn_rounded"] = crs_grouped["value_bn"].round(2)

crs_grouped.head()

,year,donor_group,value,value_bn,value_bn_rounded
0,2002,G7,4361.032566,4.361033,4.36
1,2002,Other - Country,131.898886,0.131899,0.13
2,2002,"Other - DFI, IFI, Bank",859.946212,0.859946,0.86
3,2002,UN Agency,39.043041,0.039043,0.04
4,2002,World Bank,2443.378006,2.443378,2.44


#### EXPLORING VISUALISATIONS WITH PLOTLY EXPRESS

In [147]:
grouped_dg =crs_grouped["donor_group"].unique()
donor_dropdown_options = [
    {"label": "All Sectors", "method": "update", "args": [{"visible": [True] * len(crs_grouped)}, {}]}
]
for donor in grouped_dg:
    visibility = [True if donor == group else False for group in list(crs_grouped["donor_group"].unique())]
    donor_dropdown_options.append(
        {"label": donor, "method": "update", "args": [{"visible": visibility}, {}]}
    )
fig = px.bar(
    crs_grouped,
    x="year",
    y="value_bn_rounded",
    color="donor_group",   # This separates the stacks by donor
    text_auto=True,       # Optional: shows values on bars
    title="ODA flows to Pakistan by year and donor group",
    labels={
        "year": "Year",
        "value_bn_rounded": "ODA disbursement ($billion)",
        "donor_group": "Donor Group"
    },
    color_discrete_sequence=px.colors.qualitative.Safe  # Optional: choose nice colors
)

# Customize the layout a bit
fig.update_layout(
        updatemenus=[{
        "buttons": donor_dropdown_options,
        "direction": "down",
        "showactive": True,
        "x": 1.12,  
        "y": 1.2,  
        "xanchor": "center",
        "yanchor": "top"
    }],

    barmode="stack",  # Stack the bars
    title_font=dict(family="Arial", size=16),
    xaxis_title_font=dict(family="Arial", size=12),
    yaxis_title_font=dict(family="Arial", size=12),
    legend_title="Donor",
    template="plotly_white"
)

fig.show()

### definitely need to recategorise donor names, grouping roughly as follows:
# multilaterals: ADB, WB(IDA & IBRD), UN agencies, EU institution
# bilterals: USA, UK, France, Germany, Australia, Japan, Saudi, 'other'

Bar Chart by Sector-Name and additonal column added in CRS_Data

In [148]:
# Clean column names
crs_data.columns = crs_data.columns.str.strip()
sector_category.columns = sector_category.columns.str.strip()

#Convert 'value' from million to billion
crs_data["value"] = crs_data["value"] / 1000  # 1,000 million = 1 billion
print(crs_data.columns)


Index(['donor_code', 'donor_name', 'recipient_code', 'recipient_name',
       'sector_code', 'sector_name', 'measure_code', 'measure_name',
       'channel_code', 'channel_name', 'flow_code', 'flow_name',
       'flow_type_code', 'flow_type_name', 'data_type_code', 'amount_type',
       'md_dim_name', 'md_id', 'unit_measure_code', 'unit_measure_name',
       'year', 'value', 'base_period', 'status_name', 'unit_multiplier',
       'donor_agency_code', 'crs_id', 'project_id', 'region_code',
       'income_group_code', 'channel_delivery_code', 'channel_delivery_name',
       'category_code', 'category_name', 'finance_type_code',
       'finance_type_name', 'bi_multi_code', 'short_description',
       'project_title', 'long_description', 'sdg_focus', 'gender_equality',
       'environment', 'governance', 'rmnch', 'drr', 'nutrition', 'disability',
       'biodiversity', 'climate_mitigation', 'climate_adaptation',
       'desertification', 'amount_tied', 'amount_untied',
       'nature_of_su

In [153]:
# Group by year and sector-group, sum the values
grouped = crs_data.groupby(["year", "sector-group"], as_index=False)["value"].sum()

# Rename columns for clarity
grouped.columns = ["year", "sector_group", "total_value"]

# Ensure year is treated as a category for better bar spacing
grouped["year"] = grouped["year"].astype(str)

# Create bar chart
fig = px.bar(
    grouped,
    x="year",
    y="total_value",
    color="sector_group",
    text="total_value",  # Show actual values on bars
    title="ODA Budget to Pakistan by Year and Sector Group (in Billions)",
    labels={
        "year": "Year",
        "total_value": "ODA Disbursement ($Billion)",
        "sector_group": "Sector Group"
    },
    color_discrete_sequence=px.colors.qualitative.Safe
)

# Customize layout
fig.update_layout(
    barmode="stack",
    title_font=dict(family="Arial", size=16),
    xaxis_title_font=dict(family="Arial", size=12),
    yaxis_title_font=dict(family="Arial", size=12),
    legend_title="Sector Group",
    template="plotly_white"
)

# format the bar labels for better readability
fig.update_traces(texttemplate='%{text:.2f}', textposition='inside')

fig.show()

In [ ]:
# Add dropdown filter for sector groups
sector_groups = grouped["sector_group"].unique()
years = grouped["year"].unique()

# Dropdown options for sector groups
sector_dropdown_options = [
    {"label": "All Sectors", "method": "update", "args": [{"visible": [True] * len(grouped)}, {}]}
]
for sector in sector_groups:
    visibility = [True if group == sector else False for group in list(grouped["sector_group"].unique())]
    sector_dropdown_options.append(
        {"label": sector, "method": "update", "args": [{"visible": visibility}, {}]}
    )
# Dropdown options for years
year_dropdown_options = [
    {"label": "All Years", "method": "update", "args": [{"visible": [True] * len(grouped)}, {}]}
]
for year in years:
    visibility = [True if y == year else False for y in grouped["year"]]
    year_dropdown_options.append(
        {"label": year, "method": "update", "args": [{"visible": visibility}, {}]}
    )

# Add both dropdown filters to layout


fig.update_layout(
    updatemenus=[{
            "buttons": sector_dropdown_options,
            "direction": "down",
            "showactive": True,
            "x": 1,  # Position sector filter at top-left
            "y": 1.2,
            "xanchor": "center",
            "yanchor": "top",
            "name": "Sector Group"
        },
        {
            "buttons": year_dropdown_options,
            "direction": "down",
            "showactive": True,
            "x": 1.35,  # Position year filter at top-right
                        "y": 1.2,
            "xanchor": "center",
            "yanchor": "top",
            "name": "Year"
        }

],
    barmode="stack",
    title_font=dict(family="Arial", size=16),
    xaxis_title_font=dict(family="Arial", size=12),
    yaxis_title_font=dict(family="Arial", size=12),
    legend_title="Sector Group",
    template="plotly_white"
)

# Format bar labels for readability
fig.update_traces(texttemplate='%{text:.2f}', textposition='inside')

# Show figure
fig.show()

In [151]:
print(crs_data[["year", "sector-group", "value"]].head())

   year sector-group     value
0  2020       Health  0.000007
1  2020       Health  0.000006
2  2017       Health  0.000154
3  2017       Health  0.000129
4  2017       Health  0.000538
